# Deep Learning based FACs and Action Unit occurrence and Intensity Estimation

#### \- Amogh Gudi, Emrah Tasli, Tim den Uyl, Andreas Marulis
https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7284873

### Plan
1. Load dataset and labels
2. Build a model
3. Train Model
4. Test Model

**Importing Libraries**

In [1]:
import keras
import numpy as np

/home/amogh/anaconda3/envs/faus_dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

### 1. Loading dataset and labels

a. Loading dataset

In [4]:
BP4D_base_path='/home/amogh/cmu/dataset/BP4D/'

In [5]:
BP4D_training_folder=BP4D_base_path+'BP4D-training/'

In [6]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
gen=ImageDataGenerator()

Batches from directory:

In [8]:
batches_from_dir=gen.flow_from_directory(BP4D_training_folder,target_size=(48,48))

Found 553 images belonging to 1 classes.


In [9]:
type(batches_from_dir)

keras.preprocessing.image.DirectoryIterator

b. Loading labels

Function to get labels according to subject, sequence, frame No: (details and examples in datasets.ipynb)

In [12]:
def label_getter(subj_req,seq_req,frame_req):
    au_file_reqd=BP4D_AU_path+f'{subj_req}_{seq_req}.csv'
    print (au_file_reqd)
    df_reqd=pd.read_csv(au_file_reqd)
    list_of_faus=list(df_reqd.loc[df_example['0']==frame_req].iloc[0]) #list: [frame_no, fau1, fau2, fau3....]
    #choose FAUS for which you want the labels.
    list_mask=[1,2,4,6,7,10,12,14,15,17,23]
    list_final=[list_of_faus[i] for i in list_mask]
    return list_final

In [13]:
label_getter('F001','T1',2440)

/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv


[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]

We define our own flow_from_directory function so that we can assign multiple labels to the training data before passing it to the training function

In [14]:
def multiclass_flow_from_directory(batches_from_dir, label_getter):
    for x, y in batches_from_dir:
        yield x, label_getter('F001','T1',2440)

In [15]:
gen2=multiclass_flow_from_directory(batches_from_dir,label_getter)

In [ ]:
count=0
for obj in gen2:
    if count<5:
        print (obj[1])
    count+=1

/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv
/hom

In [ ]:
BP4D_training_folder+batches_from_dir.filenames[23]

In [ ]:
plt.imread(BP4D_training_folder+batches_from_dir.filenames[23])

In [43]:
keras.utils.to_categorical((3,4,4,1),6)

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.]])

### 2. Building a Model

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D,Flatten,Dropout
from keras.optimizers import SGD

In [34]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(5,5),strides=(1,1),activation='relu',input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(64,kernel_size=(5,5),strides=(1,1),activation='relu'))
model.add(Conv2D(128,kernel_size=(4,4),strides=(1,1),activation='relu'))
model.add(Flatten())
model.add(Dense(3072,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6,activation='sigmoid'))

In [42]:
model.compile(loss='binary_crossentropy',optimizer=SGD(lr=0.01,momentum=0.9))

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 17, 17, 64)        102464    
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 14, 14, 128)       131200    
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 3072)              77073408  
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 18438     
Total para

### 3. Training the model

### 4. Testing the model